# Image Colorization Final Project
Authors: Aret Tinoco, Keshav Gupta, Hal Halberstadt

Dataset: https://www.kaggle.com/datasets/darthgera/colorization

---

Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys
from pathlib import Path
from PIL import Image # for resizing images

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid

import tensorflow as tf
from tensorflow.keras import models, layers, Input, Model, callbacks
from tensorflow.keras.layers import Lambda
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from IPython.display import display, HTML

Ease of reading

In [2]:
pd.set_option('display.max_columns', 200)
pd.options.display.width = 120
pd.options.display.max_colwidth = 50
display(HTML("<style>.container { width:100% !important; }</style>"))

Now to state the directory of the data to retrieve from.

In [3]:
data_dir = Path("C:/Users/smhal/Desktop/archive2") 
img_shape = (512, 512, 3)

folder_paths = ['color', 'bw', 'color_val', 'bw_val']

Generators

The first generator gets 

In [ ]:
class first_n(object): # adapted from https://wiki.python.org/moin/Generators

    def __init__(self, n):
        self.n = n
        self.num = 0

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        if self.num < self.n:
            cur, self.num = self.num, self.num+1
            return cur
        raise StopIteration()


In [ ]:
def Xtest_gen():
    '''Generator for test data
    converts RBG image to HSL numpy array'''
    test_max = 4500
    
    subset_dir = data_dir / folder_paths[3]
    pics = list(subset_dir.glob('*.jpg'))
    pic = np.random.randint(test_max)
    img = load_img(pic)
    
    X = np.zeros((img_shape[0], img_shape[1], img_shape[2]), dtype='float32')
    
    yield X

In [ ]:
def ytest_gen():
    '''Generator for test data
    converts RBG image to HSL numpy array'''
    test_max = 4500
    
    subset_dir = data_dir / folder_paths[2]
    pics = list(subset_dir.glob('*.jpg'))
    pic = np.random.randint(test_max)
    img = load_img(pic)
    
    X = np.zeros((img_shape[0], img_shape[1], img_shape[2]), dtype='float32')
    
    yield X

In [ ]:
def Xtrain_gen():
    '''Generator for training data
    converts image from 3 channel B/W to 1 channel'''
    test_max = 18000
    
    subset_dir = data_dir / folder_paths[1]
    pics = list(subset_dir.glob('*.jpg'))
    img = load_img(pic)
    pic = np.random.randint(test_max)
    img = load_img(pic)
    
    X = np.zeros((img_shape[0], img_shape[1], img_shape[2]), dtype='float32')
    
    yield X[0]

In [ ]:
def ytrain_gen():
    '''Generator for training data
    converts image from 3 channel B/W to 1 channel'''
    test_max = 18000
    
    subset_dir = data_dir / folder_paths[0]
    pics = list(subset_dir.glob('*.jpg'))
    img = load_img(pic)
    pic = np.random.randint(test_max)
    img = load_img(pic)
    
    X = np.zeros((img_shape[0], img_shape[1], img_shape[2]), dtype='float32')
    
    yield X[0]

In [9]:
def read_folder(subset_name, data_dir, is_color=False):
    subset_dir = data_dir / subset_name
    pics = subset_dir.glob('*.jpg')
    num_pics = int(len(list(pics))/15)
    print(f"#imgs {num_pics}, folder '{subset_name}'")
    
    X = np.zeros((num_pics, img_shape[0], img_shape[1], img_shape[2]), dtype='float32')
    
    for i, pic in enumerate(subset_dir.glob('*.jpg')):
        if i < num_pics:
            img = load_img(pic)
            X[i] = img
    
    return (X / 255)

Next I want to read the data from the files and then just to make sure I am getting the right data from the right files

In [ ]:
K.clear_session()  # delete old models
act_fun='relu'

inputs=Input(img_shape)

# YOUR CONVOLUTIONAL LAYERS GO HERE
filters=3

x = layers.SeparableConv2D(filters, 2, padding='same', activation=act_fun)(inputs)
x = layers.SeparableConv2D(filters, 2, padding='same', activation=act_fun)(inputs)
x = layers.SeparableConv2D(filters, 2, padding='same', activation=act_fun)(inputs)
x = layers.SeparableConv2D(filters, 2, padding='same', activation=act_fun)(inputs)
# x = layers.MaxPooling2D(2, padding='same', strides=1)(x)

# Flatten and output Block
embedding_model = Model(inputs, x)

In [ ]:
embedding_model.summary()

In [ ]:
# input0 = Input(img_shape, name='input0')
# input1 = Input(img_shape, name='input1')
# input2 = Input(img_shape, name='input2')

# sub0 = embedding_model(input0)
# sub1 = embedding_model(input1)
# sub2 = embedding_model(input2)

# pred = layers.Concatenate(axis=1)([sub0, sub1, sub2])

# model = Model([input0, input1, input2], pred)

# model.compile(optimizer='rmsprop', loss='mse',  metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
# model_input = datagen.flow(X_train, y_train, batch_size=32)

embedding_model.compile(optimizer='rmsprop', loss='mse',  metrics=['loss'])
embedding_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

END